In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import skimage

In [4]:
#transferLearning

In [ ]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [6]:

# Root directory of the project

ROOT_DIR = "C:\\Users\\Nutzer\\anaconda3\\envs\\myenv\\Mask_RCNN (works)"



# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
 
# Path to trained weights file #transfer learning: 
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5") #we only needs its structure

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [7]:
class CustomConfig(Config):
    # Give the configuration a recognizable name
    NAME = "object"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 4 #batch size

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + organoid

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 5 #one step = batch size

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [8]:
############################################################
#  Dataset
############################################################

class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        
        # Add classes. We have only one class to add.
        self.add_class("object", 1, "nucleus")


        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # We mostly care about the x and y coordinates of each region #annotation of train folder
        if subset == 'train':
            annotations1 = json.load(open('dataset\\nucleus_jpg\\train\\via_region_data.json'))
        elif subset == 'val':
            annotations1 = json.load(open('dataset\\nucleus_jpg\\val\\via_region_data.json'))
        else:
            print("invalid subset")
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        '''
        print("check**********************************************************")
        for ann in annotations:
            print("ann type: ", type(ann))
            print("ann val: ", ann)
            print("************************************")
        print("checkEnd*******************************************************")
        '''
        
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
            '''
            print("check**********************************************************")
            print("a: " , a)
            
            for r in a['regions']:
                print("------------")
                print("a[regions] type: ", type(a['regions']))
                print("a[regions]: ", a['regions'])
                
                print("------------")
                print("a[regions][r]: ", a['regions'][r])
                
                print("a[regions][r]['shape_attributes']: ", a['regions'][r]['shape_attributes'])
                
                #print("------------")
                #print("r['0'] type: ", type(r[0]) )
                #print("r['0'] val: ", r[0])
                
                #print("------------")
                print("************************************")
            print("checkEnd*******************************************************")
            '''
            
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            
            
            
            
            #polygons = [a['regions'][r]['shape_attributes'] for r in a['regions']]
            
            polygons = [r['shape_attributes'] for r in a['regions']] 
            
            #objects = [a['regions'][s]['region_attributes']['label'] for s in a['regions']]
            
            objects = [s['region_attributes']['type'] for s in a['regions']]
            
            #print("objects:",objects)
            
            
            name_dict = {"nucleus": 1}
            
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
    
            image_path = os.path.join(dataset_dir, a['filename'])
            #print("check***************************************")
            #print("a[filename]: ", a['filename'])
            #print("checkEnd***********************************")
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [9]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom("dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom("dataset", "val")
    dataset_val.prepare()

    
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the head layers
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=60,
                layers='heads')
    

        
    return model

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])



In [ ]:
model = train(model)


In [ ]:
new_history = model.keras_model.history.history


In [ ]:
#training evaluation:

In [ ]:
EPOCHS = []
for i in range(60):
    EPOCHS.append(i+1)
    

history = new_history

In [ ]:
EPOCHS = [1:]
history = new_history

In [ ]:
epochs = range(EPOCHS[-1])

In [ ]:
epochs

In [ ]:
epochs = range(EPOCHS[-1])

plt.figure(figsize=(18, 6))

plt.subplot(131)
plt.plot(epochs, history['loss'], label="train loss")
plt.plot(epochs, history['val_loss'], label="valid loss")
plt.legend()
plt.subplot(132)
plt.plot(epochs, history['mrcnn_class_loss'], label="train class loss")
plt.plot(epochs, history['val_mrcnn_class_loss'], label="valid class loss")
plt.legend()
plt.subplot(133)
plt.plot(epochs, history['mrcnn_mask_loss'], label="train mask loss")
plt.plot(epochs, history['val_mrcnn_mask_loss'], label="valid mask loss")
plt.legend()

plt.show()